# 🚀 Quick Start

- Run **Cell → Run All** in the Jupyter menu.  
- This will:
  - Load functions and UI widgets  
  - Show the interactive panel  
  - Generate example demo spectra/gels automatically  
- Exported images will appear in `exports/uvvis/` and `exports/gels/`.


In [ ]:
from IPython.display import IFrame, FileLink, display
import ipywidgets as widgets

print("📘 Teacher’s Workbook preview:")
display(IFrame("Teachers_Workbook_UVVis_Electrophoresis.pdf", width=800, height=600))
display(FileLink("Teachers_Workbook_UVVis_Electrophoresis.pdf", result_html_prefix="⬇️ Download: "))

print("📗 Student’s Workbook preview:")
display(IFrame("Students_Workbook_UVVis_Electrophoresis.pdf", width=800, height=600))
display(FileLink("Students_Workbook_UVVis_Electrophoresis.pdf", result_html_prefix="⬇️ Download: "))


# UV-Visible Spectrometry Simulation

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import os

# Ensure export folder exists
os.makedirs("exports/uvvis", exist_ok=True)

# Beer-Lambert law absorbance simulation
def gaussian(x, mu, eps, sigma):
    return eps * np.exp(-(x-mu)**2/(2*sigma**2))

def beer_lambert_absorbance(wavelengths, mixture, path_cm=1.0):
    A = np.zeros_like(wavelengths, dtype=float)
    for peaks, conc in mixture.items():
        for (mu, eps, sigma) in peaks:
            A += conc * path_cm * gaussian(wavelengths, mu, eps, sigma)
    return A

def detect_peaks(wavelengths, absorbance, threshold=0.05, distance=5):
    peaks = []
    for i in range(1, len(absorbance)-1):
        if absorbance[i] > threshold and absorbance[i] > absorbance[i-1] and absorbance[i] > absorbance[i+1]:
            if len(peaks)==0 or (wavelengths[i]-peaks[-1][0]) >= distance:
                peaks.append((wavelengths[i], absorbance[i]))
    return peaks

def export_uvvis_png(wavelengths, absorbance):
    ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    fname = f"exports/uvvis/uvvis_spectrum_{ts}.png"
    plt.figure()
    plt.plot(wavelengths, absorbance, lw=2)
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Absorbance (AU)")
    plt.title("UV-Vis Spectrum")
    plt.savefig(fname, dpi=150)
    plt.close()
    print(f"✅ Exported spectrum to {fname}")


In [ ]:

def uvvis_ui():
    # Molecule library with extended biochemicals
    molecules = {
        "ADN (260 nm)": [(260, 6600, 15)],
        "Protéine (280 nm)": [(280, 5500, 15)],
        "Nitrite (354 & 371 nm)": [(354, 1200, 15), (371, 1500, 15)],
        "Rouge (450 nm)": [(450, 20000, 20)],
        "Vert (520 nm)": [(520, 25000, 20)],
        "Bleu (620 nm)": [(620, 22000, 20)],
    }
    molecules.update({
        "NADH (260 & 340 nm)": [(260, 30000, 15), (340, 6200, 25)],
        "NAD+ (260 nm)": [(260, 18000, 15)],
        "ATP (260 nm)": [(260, 15000, 15)],
        "Chlorophylle a (430 & 662 nm)": [(430, 40000, 20), (662, 70000, 20)],
        "Chlorophylle b (453 & 642 nm)": [(453, 35000, 20), (642, 60000, 20)],
        "β-Carotène (450 nm)": [(450, 140000, 25)],
        "Hème (414, 540, 575 nm)": [(414, 120000, 10), (540, 11000, 15), (575, 8000, 15)],
        "Flavine (370 & 450 nm)": [(370, 11000, 20), (450, 12500, 20)],
        "Rouge de phénol (430 & 560 nm)": [(430, 15000, 15), (560, 18000, 20)]
    })

    # Widgets
    mol1 = widgets.Dropdown(options=list(molecules.keys()), description="Espèce 1")
    conc1 = widgets.FloatSlider(value=50, min=0, max=200, step=1, description="[1] (µM)")
    mol2 = widgets.Dropdown(options=list(molecules.keys()), description="Espèce 2")
    conc2 = widgets.FloatSlider(value=0, min=0, max=200, step=1, description="[2] (µM)")

    lmin = widgets.IntSlider(value=220, min=200, max=700, step=1, description="λ min")
    lmax = widgets.IntSlider(value=400, min=200, max=800, step=1, description="λ max")
    step = widgets.IntSlider(value=1, min=1, max=10, description="Pas (nm)")
    path = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="Chemin (cm)")

    baseline = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="Baseline")
    noise = widgets.FloatSlider(value=0.0, min=0.0, max=0.1, step=0.01, description="Bruit (ET)")
    correct_baseline = widgets.Checkbox(value=False, description="Correction baseline")

    peak_thresh = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description="Seuil pics (A)")

    export_btn = widgets.Button(description="Export PNG", button_style="success")

    output = widgets.Output()

    def update_plot(change=None):
        with output:
            output.clear_output()
            wavelengths = np.arange(lmin.value, lmax.value, step.value)
            mixture = {}
            if conc1.value>0:
                mixture[tuple(molecules[mol1.value])] = conc1.value*1e-6
            if conc2.value>0:
                mixture[tuple(molecules[mol2.value])] = conc2.value*1e-6
            A = beer_lambert_absorbance(wavelengths, mixture, path_cm=path.value)
            A += baseline.value
            if noise.value>0:
                A += np.random.normal(0, noise.value, size=A.shape)
            if correct_baseline.value:
                A -= np.linspace(A[0], A[-1], len(A))
            plt.figure(figsize=(6,4))
            plt.plot(wavelengths, A, lw=2)
            plt.xlabel("Wavelength (nm)")
            plt.ylabel("Absorbance (AU)")
            plt.title("Simulated UV-Vis Spectrum")
            plt.grid(True, ls=":")
            plt.show()

            # Peaks
            peaks = detect_peaks(wavelengths, A, threshold=peak_thresh.value)
            if peaks:
                print("Detected peaks:")
                for wl, ab in peaks:
                    print(f" - {wl:.0f} nm : {ab:.3f} AU")

    def export_action(b):
        wavelengths = np.arange(lmin.value, lmax.value, step.value)
        mixture = {}
        if conc1.value>0:
            mixture[tuple(molecules[mol1.value])] = conc1.value*1e-6
        if conc2.value>0:
            mixture[tuple(molecules[mol2.value])] = conc2.value*1e-6
        A = beer_lambert_absorbance(wavelengths, mixture, path_cm=path.value)
        A += baseline.value
        if noise.value>0:
            A += np.random.normal(0, noise.value, size=A.shape)
        if correct_baseline.value:
            A -= np.linspace(A[0], A[-1], len(A))
        export_uvvis_png(wavelengths, A)

    mol1.observe(update_plot, names="value")
    mol2.observe(update_plot, names="value")
    conc1.observe(update_plot, names="value")
    conc2.observe(update_plot, names="value")
    lmin.observe(update_plot, names="value")
    lmax.observe(update_plot, names="value")
    step.observe(update_plot, names="value")
    path.observe(update_plot, names="value")
    baseline.observe(update_plot, names="value")
    noise.observe(update_plot, names="value")
    correct_baseline.observe(update_plot, names="value")
    peak_thresh.observe(update_plot, names="value")
    export_btn.on_click(export_action)

    ui = widgets.VBox([
        widgets.HBox([mol1, conc1]),
        widgets.HBox([mol2, conc2]),
        widgets.HBox([lmin, lmax, step]),
        path,
        widgets.HBox([baseline, noise]),
        correct_baseline,
        peak_thresh,
        export_btn
    ])

    display(ui, output)
    update_plot()


### 🧪 Try it!
- Select biomolecules from the dropdowns (e.g. *NADH + Protéine*, *Chlorophylle a + b*).  
- Adjust concentrations to see overlapping spectra.  
- Use **Baseline correction** + **Peak detection** for analysis.  
- Export spectra with the **Export PNG** button (files saved to `exports/uvvis/`).

In [ ]:
uvvis_ui()

## 📊 Demo Spectrum

In [ ]:
# --- Demo: NADH spectrum at 340 nm ---
wavelengths = np.arange(220, 400, 1)
mixture = {((260, 30000, 15), (340, 6200, 25)): 50e-6}  # NADH 50 µM
A = beer_lambert_absorbance(wavelengths, mixture, path_cm=1.0)
plt.figure(figsize=(6,4))
plt.plot(wavelengths, A, lw=2)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Absorbance (AU)")
plt.title("Demo Spectrum: NADH (50 µM)")
plt.grid(True, ls=":")
plt.show()

export_uvvis_png(wavelengths, A)
